In [ ]:
!pip install -q transformers

In [ ]:
#@title Draft V.3
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Загрузка модели и токенизатора
model_name = "cointegrated/rubert-tiny2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def get_sentence_vector(sentence):
    # Токенизация предложения
    inputs = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True)

    # Получение выходов модели
    with torch.no_grad():
        outputs = model(**inputs)

    # Использование среднего значения последнего скрытого состояния для получения вектора предложения
    sentence_vector = outputs.last_hidden_state.mean(dim=1).numpy()

    return sentence_vector


In [ ]:
# Загрузка датасета
dataset_path = "//content/drive/MyDrive/Стажировка КИА/Диалоги/database.md"
# https://drive.google.com/file/d/1LNgIg5In2nL-WGzm3kYRHUFB_i6pCPUL/view?usp=sharing

# Чтение файла с расширением .md и разделение текста на диалоги по символу ##
with open(dataset_path, 'r') as file:
    data = file.read().split('##')
    #data = file.read()

# Создание DataFrame из списка диалогов
df_manager = pd.DataFrame(data, columns=['dialog'])

# Создание векторов для каждого диалога
df_manager['vector'] = df_manager['dialog'].apply(get_sentence_vector)

# Измерение косинусного сходства между векторами
similarities = cosine_similarity(np.concatenate(df_manager['vector'].values))


In [ ]:
# Отбор диалогов по близости смысла
threshold = 0.8
selected_dialogs = []

selected = [(df_manager.iloc[i]['dialog'], df_manager.iloc[j]['dialog'])
            for i, row in enumerate(similarities)
            for j, value in enumerate(row) if i != j and value > threshold]
selected_dialogs = selected

# Запись результатов в файл
with open('sep_dialog.txt', 'w') as file:
    file.write(f"Количество диалогов с близостью {threshold*100}%: {len(selected_dialogs)}\n")
    for pair in selected_dialogs:
        file.write(f"{pair[0]} - {pair[1]}\n")
